In [ ]:
!pip uninstall spacy

In [ ]:
!pip install spacy==3.0.0

In [1]:
# spacy version 2.3.0 for French
import spacy
import json
from tqdm import tqdm

In [ ]:
!python -m spacy download en

In [ ]:
!python -m spacy download ru_core_news_sm

In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/R&D

/content/drive/MyDrive/Colab Notebooks/R&D


In [ ]:
ranges = "20-22,29-31"
for range in ranges.split(','):
    

In [6]:
def preprocess(infile):
  with open(infile, encoding='utf-8') as data_file:
    data = json.loads(data_file.read()) 
  pairs = []
  for dictionary in tqdm(data):
    pair_id = dictionary['id']
    target_lemma = dictionary['lemma']
    target_POS = dictionary['pos']
    language_1 = dictionary['id'].split('-')[0][-2:]
    language_2 = dictionary['id'].split('-')[1][:2]
    sentence_1 = dictionary['sentence1']
    sentence_2 = dictionary['sentence2']

    ranges1 = dictionary['ranges1']
    ranges2 = dictionary['ranges2']

    target_1 = ''
    for range in ranges1.split(','):
        start_1 = int(range.split('-')[0])
        end_1 = int(range.split('-')[1])
        target_1 += sentence_1[start_1:end_1]

    target_2 = ''
    for range in ranges2.split(','):
        start_2 = int(range.split('-')[0])
        end_2 = int(range.split('-')[1])
        target_2 += sentence_2[start_2:end_2]
    
    pairs.append((pair_id, target_lemma, target_POS, language_1, language_2, sentence_1, sentence_2, target_1, target_2))
  return pairs

In [7]:
def pairs_to_conllu(pairs, outfile):
  result = open(outfile, "a")
  for pair in tqdm(pairs):
    pair_id = pair[0]
    target_lemma = pair[1]
    target_POS = pair[2]
    language_1 = pair[3]
    language_2 = pair[4]
    sentence_1 = pair[5]
    sentence_2 = pair[6]
    target_1 = pair[7]
    target_2 = pair[8]

    # write comment of pair_id
    result.write('#\tpair_id\t=\t')
    result.write(pair_id)
    result.write('\n')
    # write comment of target
    result.write('#\ttarget_lemma\t=\t')
    result.write(target_lemma)
    result.write('\n')
    result.write('#\ttarget_POS\t=\t')
    result.write(target_POS)
    result.write('\n')
    
    result.write('\n')
    # write comment of first sentence
    result.write('#\tfirst_sentence\t=\t')
    result.write(sentence_1)
    result.write('\n')
    # write one token per line of first text
    if language_1 != 'zh' and language_1 != 'ru':
      nlp_1 = spacy.load(language_1+'_core_web_sm')
    elif language_1 == 'zh':
      nlp_1 = spacy.load(language_1+'_core_web_sm')
    else:
      nlp_1 = spacy.load(language_1+'_core_news_sm')
    doc_1 = nlp_1(sentence_1)
    count = 0
    for token in doc_1:
      result.write(str(count)) # INDEX
      result.write('\t')
      result.write(token.text) # FORM
      result.write('\t')
      result.write(token.lemma_) # LEMMA
      result.write('\t')
      result.write(token.pos_) # UPOS
      result.write('\t')
      result.write(token.head.text) # HEAD
      result.write('\t')
      result.write(token.dep_) # DEPREL
      result.write('\t')
      result.write(str(token.text==target_1)) # IS TARGET
      result.write('\n')
      count += 1
    
    result.write('\n')
    # write comment of second sentence
    result.write('#\tsecond_sentence\t=\t')
    result.write(sentence_2)
    result.write('\n')
    # write one token per line of first text
    if language_2 != 'zh' and language_2 != 'ru':
      nlp_2 = spacy.load(language_2+'_core_web_sm')
    elif language_2 == 'zh':
      nlp_2 = spacy.load(language_2+'_core_web_sm')
    else:
      nlp_2 = spacy.load(language_2+'_core_news_sm')
    doc_2 = nlp_2(sentence_2)
    count = 0
    for token in doc_2:
      result.write(str(count)) # INDEX
      result.write('\t')
      result.write(token.text) # FORM
      result.write('\t')
      result.write(token.lemma_) # LEMMA
      result.write('\t')
      result.write(token.pos_) # UPOS
      result.write('\t')
      result.write(token.head.text) # HEAD
      result.write('\t')
      result.write(token.dep_) # DEPREL
      result.write('\t')
      result.write(str(token.text==target_2)) # IS TARGET
      result.write('\n')
      count += 1
    result.write('\n\n')
  result.close()

In [ ]:
en_zh_pairs = preprocess('test.en-zh.data')
pairs_to_conllu(en_zh_pairs, 'test.en-zh.conllu')

100%|██████████| 1000/1000 [31:20<00:00,  1.88s/it]


In [11]:
en_ru_pairs = preprocess('test.en-ru.data')
pairs_to_conllu(en_ru_pairs, 'test.en-ru.conllu')

100%|██████████| 1000/1000 [32:23<00:00,  1.94s/it]


In [8]:
def find_info(sentence_pair):
  basic_info, sent_1, sent_2 = sentence_pair.split('\n\n')
  info = []

  for line in basic_info.split('\n'):
    breakdown = line.split('\t')
    if breakdown[1] == 'pair_id':
      pair_id = breakdown[3]
  
  info.append(pair_id)
  
  for sent in (sent_1, sent_2):
    target = []
    head = []
    dependent = []
    for line in sent.split('\n'):
      breakdown = line.split('\t')
      if len(breakdown) == 4:
        info.append(breakdown[-1])
      elif (len(breakdown) > 4) and breakdown[6] == 'True':
        target.append(breakdown[1])
        head.append(breakdown[4])

    # Find dependent  
    for line in sent.split('\n'):
      breakdown = line.split('\t')
      if (len(breakdown) > 4) and breakdown[4] in target:
        dependent.append(breakdown[1])
    
    info.append(target)
    info.append(head)
    info.append(dependent)
  return info

In [9]:
def conllu_to_info(conllu_file, gold_file):
  #language_pair = conllu_file.split('.')[1]
  prefix = conllu_file[:-7]
  
  with open(gold_file, encoding='utf-8') as label_file:
    data = json.loads(label_file.read())

  labels = []
  for dictionary in tqdm(data):
    if dictionary['tag'] == 'T':
      labels.append(1)
    else:
      labels.append(0)
  con_file = open(conllu_file, 'r')
  infos = open(prefix+'.info', "a", encoding='utf8')
  i = 0
  for sentence_pair in tqdm(con_file.read().split('\n\n\n')[:-1]):
    info = find_info(sentence_pair)
    info.append(labels[i])
    #infos.write(json.dumps(info))
    json.dump(info, infos, ensure_ascii=False)
    infos.write('\n')
    i += 1

In [10]:
def conllu_to_info_test(conllu_file):
  #language_pair = conllu_file.split('.')[1]
  prefix = conllu_file[:-7]
  
  #with open(gold_file, encoding='utf-8') as label_file:
  #  data = json.loads(label_file.read())

  #labels = []
  #for dictionary in tqdm(data):
  #  if dictionary['tag'] == 'T':
  #    labels.append(1)
  #  else:
  #    labels.append(0)
  con_file = open(conllu_file, 'r')
  infos = open(prefix+'.info', "a", encoding='utf8')
  i = 0
  for sentence_pair in tqdm(con_file.read().split('\n\n\n')[:-1]):
    info = find_info(sentence_pair)
    #info.append(labels[i])
    #infos.write(json.dumps(info))
    json.dump(info, infos, ensure_ascii=False)
    infos.write('\n')
    i += 1

In [ ]:
#conllu_to_info_test('test.en-zh.conllu')

100%|██████████| 1000/1000 [00:00<00:00, 9215.19it/s]


In [13]:
conllu_to_info('test.en-ru.conllu', 'test.en-ru.gold')

100%|██████████| 1000/1000 [00:00<00:00, 7887.66it/s]
